In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
loaded = np.load('/content/drive/MyDrive/msc_experiments/in10_split_converted.npz', allow_pickle=True)
x_train = loaded['x_train']
y_train = loaded['y_train']
x_test = loaded['x_test_none']
y_test = loaded['y_test']
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((10000, 5096), (10000,), (3000, 5096), (3000,))

In [4]:
N, D_in, H1, H2, H3, D_out = x_train.shape[0], x_train.shape[1], 2000, 1000, 500, 10
learning_rate = 0.001

x_train = torch.Tensor(x_train)
y_train = np.squeeze(torch.LongTensor(y_train))

In [5]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.Sigmoid(),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(H1, H2),
    torch.nn.Sigmoid(),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(H2, H3),
    torch.nn.ReLU(),
    torch.nn.Linear(H3, D_out),
    torch.nn.LogSoftmax(dim = 1),
).cuda()

loss_fn = torch.nn.NLLLoss()
    
losses = []

batchs = 256
    
trainset = torch.utils.data.TensorDataset(x_train, y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batchs,
                                          shuffle=True, num_workers=1)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

batchesn = int(N/batchs)
epochs = 25

In [6]:
for epoch in range(epochs):
    running_loss = 0.0
    for i, datap in enumerate(trainloader, 0):
        #print(i)
        # get the inputs
        inputs, labels = datap
        inputs = inputs.to('cuda', non_blocking=True)
        labels = labels.to('cuda', non_blocking=True)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % batchesn == batchesn-1:
            print('[epoch {}] loss: {}'.format(epoch + 1, running_loss/batchesn))
            running_loss = 0.0
    scheduler.step()
  
torch.save(model, '/content/drive/MyDrive/msc_experiments/vgg_benchamrk.pt')

[epoch 1] loss: 2.0989592381012745
[epoch 2] loss: 1.5027588911545582
[epoch 3] loss: 1.2778293964190361
[epoch 4] loss: 1.1346410650473375
[epoch 5] loss: 1.089057942231496
[epoch 6] loss: 1.038713282499558
[epoch 7] loss: 1.0151002086125887
[epoch 8] loss: 0.9659903232867901
[epoch 9] loss: 0.9754716860942352
[epoch 10] loss: 0.925364090846135
[epoch 11] loss: 0.8997087921851721
[epoch 12] loss: 0.9095994356351021
[epoch 13] loss: 0.8891634253355173
[epoch 14] loss: 0.8748933336673639
[epoch 15] loss: 0.8812622611339276
[epoch 16] loss: 0.8483840517508678
[epoch 17] loss: 0.8517190248538287
[epoch 18] loss: 0.838348144139999
[epoch 19] loss: 0.8432299723991981
[epoch 20] loss: 0.8121888087346003
[epoch 21] loss: 0.8223123718530704
[epoch 22] loss: 0.8073563377062479
[epoch 23] loss: 0.827581274203765
[epoch 24] loss: 0.7944798805774786
[epoch 25] loss: 0.765562586295299


In [7]:
model = torch.load('/content/drive/MyDrive/msc_experiments/vgg_benchamrk.pt', 
                   map_location=torch.device('cpu'))
x_test = torch.Tensor(x_test)
y_test = np.squeeze(torch.LongTensor(y_test))

model.eval()
outputs = model(x_test)
probals, predicted = torch.max(outputs, 1)

In [8]:
c = (predicted == y_test).squeeze()
correct = c.sum().item()

print('samples correct={} training accuracy={}'.format(correct, np.round(correct/x_test.shape[0], 3)))

samples correct=2020 training accuracy=0.673
